In [1]:
%matplotlib inline
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import signal, stats
from sklearn import linear_model
import sys
import warnings; warnings.filterwarnings("ignore")

from aux import get_seg
from disp import set_font_size
from my_stats import nanpearsonr

cc = np.concatenate

FPS_DN = 8.5  # sampling rate from neural data
DT_DN = 1/FPS_DN
NT_H_DN = 85
T_H_DN = np.arange(NT_H_DN)*DT_DN

FPS = 30.03  # sampling rate of behavioral data
DT = 1/FPS
NT_H = int(round(NT_H_DN*FPS/FPS_DN))
T_H = np.arange(NT_H)*DT

Load behavioral data from Coen et al 2014

In [2]:
df_behav = pd.read_csv('data/simple/c_song_f_behav.csv')
df_behav

,ID,FRAME,T,Q,S,P,F,FFV,FFA,FLS,FRS
0,0,0,0.000000,1,0,0,0,0.231,0.0000,0.159,0.077
1,0,1,0.033300,1,0,0,0,0.314,0.0385,0.025,0.044
2,0,2,0.066600,1,0,0,0,0.308,0.0025,0.029,0.011
3,0,3,0.099900,1,0,0,0,0.319,0.0020,0.016,0.016
4,0,4,0.133200,1,0,0,0,0.312,-0.0315,0.030,0.002
...,...,...,...,...,...,...,...,...,...,...,...
4978565,275,27100,902.430902,1,0,0,0,0.217,0.0280,0.029,0.058
4978566,275,27101,902.464202,1,0,0,0,0.210,-0.1045,0.138,0.322
4978567,275,27102,902.497502,1,0,0,0,0.008,-0.0910,0.034,0.260
4978568,275,27103,902.530803,1,0,0,0,0.028,0.0695,0.010,0.090


In [3]:
# split big df into dfs for individual trials
n_tr = np.max(df_behav.ID) + 1
dfs_tr = [df_behav[df_behav.ID == i] for i in range(n_tr)]

tr_lens = np.array([len(df_tr) for df_tr in dfs_tr])
tr_lens_cum = cc([[0], np.cumsum(tr_lens)])

n_t_total = np.sum(tr_lens)
assert n_t_total == tr_lens_cum[-1]
df_behav = None

# Nonlinear adaptive dynamical system neural responses

Load params from dynamical systems fits to Baker et al data

In [4]:
# neural response fits from Baker data
df_dyn_param = pd.read_csv('data/simple/neur/baker_dyn_fit_param.csv')
df_dyn_param

,EXPT_ID,LINE,FLY,ROI,ROI_USED,FIT_ERR,SUCCESS,T_0,R_0,TAU_R,TAU_A,X_S,X_P
0,0,R70G01,0,somas,False,0.275200,True,-0.000000,0.028882,6.202615,30.336758,-4.958290,-2.597432
1,1,R70G01,1,somas,False,0.186212,True,0.117647,0.025128,3.835468,21.821295,-4.163967,-1.472075
2,2,R70G01,2,somas,False,0.248414,True,-0.000000,0.055962,6.017162,15.058055,-6.248007,-2.095952
3,3,R70G01,3,somas,False,0.455963,True,-0.000000,-0.014227,6.016828,11.820576,-5.679735,-2.118245
4,4,R70G01,4,somas,False,0.229018,True,-0.000000,0.019831,6.434825,55.496220,-4.415761,-2.863647
...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,219,VT34811,0,somas,False,0.276087,True,-0.000000,0.039119,1.799608,1440.994269,-0.049828,2.494657
220,220,VT34811,0,ascending projection,False,0.037801,True,-0.000000,-0.001886,1.298841,27.643238,0.098451,2.457383
221,221,vPN1_split,0,somas,False,0.404112,True,-0.588235,-0.014702,4.151226,6.155551,8.332263,2.353217
222,222,vPN1_split,1,somas,False,1.205977,True,0.235294,-0.095665,5.924070,90.705238,2.390705,0.604586


In [5]:
expt_ids = df_dyn_param['EXPT_ID']

# get parameters
tau_rs = np.array(df_dyn_param['TAU_R'])
tau_as = np.array(df_dyn_param['TAU_A'])
x_ss = np.array(df_dyn_param['X_S'])
x_ps = np.array(df_dyn_param['X_P'])

In [6]:
# define smln & obj function
def smlt(i_s, i_p, tau_r, tau_a, x_s, x_p):
    """Simulate response to song inputs."""
    t = np.arange(len(i_s))*DT
    r = np.nan*np.zeros(len(t))
    
    r[0] = 0
    a_s, a_p = 0, 0
    
    for ct, t_ in enumerate(t[1:], 1):
        a_s += ((DT/tau_a) * (-a_s + x_s*i_s[ct]))
        a_p += ((DT/tau_a) * (-a_p + x_p*i_p[ct]))
        r[ct] = r[ct-1] + (DT/tau_r) * (-r[ct-1] + (x_s - a_s)*i_s[ct] + (x_p - a_p)*i_p[ct])
    
    return r

In [9]:
# define smln & obj function
def smlt_ppln(i_s, i_p, tau_r, tau_a, x_s, x_p):
    """Simulate response to song inputs."""
    n = len(tau_r)
    t = np.arange(len(i_s))*DT
    rs = np.nan*np.zeros((len(t), n))
    
    rs[0, :] = 0
    a_s, a_p = 0, 0
    
    for ct, t_ in enumerate(t[1:], 1):
        a_s += ((DT/tau_a) * (-a_s + x_s*i_s[ct]))
        a_p += ((DT/tau_a) * (-a_p + x_p*i_p[ct]))
        rs[ct, :] = rs[ct-1, :] + (DT/tau_r) * (-rs[ct-1, :] + (x_s - a_s)*i_s[ct] + (x_p - a_p)*i_p[ct])
    
    return rs

Reversed songs

In [8]:
SAVE_DIR_DYN_RVRS = 'data/simple/mlv_c/perturbed_song/c_baker_dyn_rvrs'
PFX_DYN_RVRS = 'mlv_c_baker_dyn_rvrs'

if not os.path.exists(SAVE_DIR_DYN_RVRS):
    os.makedirs(SAVE_DIR_DYN_RVRS)
    
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(float)[::-1]
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(float)[::-1]
        
        r = smlt(i_s, i_p, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN_RVRS, f'{PFX_DYN_RVRS}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

Scrambled songs

In [19]:
SAVE_DIR_DYN_SCRM = 'data/simple/mlv_c/perturbed_song/c_baker_dyn_scrm'
PFX_DYN_SCRM = 'mlv_c_baker_dyn_scrm'

if not os.path.exists(SAVE_DIR_DYN_SCRM):
    os.makedirs(SAVE_DIR_DYN_SCRM)
    
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    # loop over neurons
    for expt_id, tau_r, tau_a, x_s, x_p in zip(expt_ids, tau_rs, tau_as, x_ss, x_ps):
        # compute surrogate neural response
        i_s = np.array(df_tr['S']).astype(bool)
        i_p = np.array(df_tr['P'] | df_tr['F']).astype(bool)
        
        s = np.zeros(len(df_tr), dtype=int)
        s[i_s] = 1
        s[i_p] = 2
        
        s_scrambled = s[np.random.permutation(len(s))]
        
        i_s_scrambled = (s_scrambled == 1).astype(float)
        i_p_scrambled = (s_scrambled == 2).astype(float)
        
        r = smlt(i_s_scrambled, i_p_scrambled, tau_r, tau_a, x_s, x_p)
        
        df_tr_nrl_dyn[f'R_{expt_id}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN_SCRM, f'{PFX_DYN_SCRM}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

Individual bouts reversed

In [15]:
from aux import get_seg

SAVE_DIR_DYN_BT_RVRS = 'data/simple/mlv_c/perturbed_song/c_baker_dyn_bt_rvrs'
PFX_DYN_BT_RVRS = 'mlv_c_baker_dyn_bt_rvrs'

if not os.path.exists(SAVE_DIR_DYN_BT_RVRS):
    os.makedirs(SAVE_DIR_DYN_BT_RVRS)
    
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    
    # compute surrogate neural response
    i_s = np.array(df_tr['S']).astype(bool)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(bool)

    song = np.zeros(len(df_tr), dtype=int)
    song[i_s] = 1
    song[i_p] = 2

    bt_bds = get_seg(song > 0, min_gap=1)[1]

    song_bt_rvrs = np.zeros(len(df_tr), dtype=int)

    for lb, ub in bt_bds:
        song_bt_rvrs[lb:ub] = song[lb:ub][::-1]

    i_s_bt_rvrs = (song_bt_rvrs == 1).astype(float)
    i_p_bt_rvrs = (song_bt_rvrs == 2).astype(float)
    
    rs = smlt_ppln(i_s_bt_rvrs, i_p_bt_rvrs, tau_rs, tau_as, x_ss, x_ps)
    for cr, r in enumerate(rs.T):
        df_tr_nrl_dyn[f'R_{cr}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN_BT_RVRS, f'{PFX_DYN_BT_RVRS}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................

Sine and pulse swapped

In [16]:
SAVE_DIR_DYN_MODE_SWAP = 'data/simple/mlv_c/perturbed_song/c_baker_dyn_mode_swap'
PFX_DYN_MODE_SWAP = 'mlv_c_baker_dyn_mode_swap'

if not os.path.exists(SAVE_DIR_DYN_MODE_SWAP):
    os.makedirs(SAVE_DIR_DYN_MODE_SWAP)
    
# compute surrogate neural responses for each trial
for ctr, df_tr in enumerate(dfs_tr):
    df_tr_nrl_dyn = df_tr.copy()
    sys.stdout.write('.')
    
    i_s = np.array(df_tr['S']).astype(bool)
    i_p = np.array(df_tr['P'] | df_tr['F']).astype(bool)
    
    rs = smlt_ppln(i_p, i_s, tau_rs, tau_as, x_ss, x_ps)
    for cr, r in enumerate(rs.T):
        df_tr_nrl_dyn[f'R_{cr}'] = r
        
    np.save(os.path.join(SAVE_DIR_DYN_MODE_SWAP, f'{PFX_DYN_MODE_SWAP}_tr_{ctr}.npy'), np.array([{'df': df_tr_nrl_dyn}]))

....................................................................................................................................................................................................................................................................................